In [71]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.metrics import classification_report

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Josvaldes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Josvaldes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [72]:
# Leer el archivo CSV
df = pd.read_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/textMining/proyecto/TextMining/TextMining/RECLAMOS 3.csv', encoding='utf-8')
df.head()

,problema_id,descripcion,alias_8_nombre3,problema_n,obsitem,Concaobsitem
0,70e03aaf-6cfb-42e8-92ae-fcb46ff479dd,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.),GN - Gas Natural,B1 - Pérdidas de gas con piezas dañadas,HAY UNA PERILLA DE LAS HORNALLA QUE NO SE PUED...,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
1,e5be47a0-c506-4a79-87e9-2a45f06d496c,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.),GN - Gas Natural,G - Perilla no gira / gira mal,HAY UNA PERILLA DE LAS HORNALLA QUE NO SE PUED...,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
2,2b2af341-20c5-4607-baa0-01972617647e,TERMO ELECTRICO EL-55,ELE - Termo eléctrico,D - No enciende,DESPUÉS DE QUE UNA PERSONA SE DUCHO NO VOLVIÓ ...,"TERMO ELECTRICO EL-55 ELE - Termo eléctrico "" ..."
3,ab0ed03a-8aab-4317-9aad-fcc06dd41520,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.),GN - Gas Natural,D2 - Hornalla Mal funcionamiento,NaN,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
4,ab0ed03a-8aab-4317-9aad-fcc06dd41520,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.),GN - Gas Natural,D2 - Hornalla Mal funcionamiento,LAS HORNALLAS TARDAN MUCHÍSIMO TIEMPO EN PREND...,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...


In [73]:
#Dimenciones del dataset
df.shape

(18714, 6)

In [75]:
# Cuenta las cantidades de clases
clase_counts = df["problema_n"].value_counts()

# Obtiene la lista de clases únicas
clases_unicas = df["problema_n"].unique()

# Cuenta la cantidad de clases únicas
cantidad_clases = len(clases_unicas)

# Imprime la cantidad de clases únicas
print("Cantidad total de clases únicas:", cantidad_clases)

Cantidad total de clases únicas: 51


In [77]:
# Obtén un resumen de estadísticas descriptivas para columnas numéricas
resumen_estadistico = df.describe(include='all')
resumen_estadistico

,problema_id,descripcion,alias_8_nombre3,problema_n,obsitem,Concaobsitem
count,18714,18698,18698,18714,16392,18714
unique,51,51,5,51,12259,12945
top,8c5fec8e-37bc-46ac-a031-717476ef2f6f,TERMO ELECTRICO EL-55,GN - Gas Natural,E - Puerta de horno - No cierra / Se cae,NO CALIENTA,"COCINA CANDOR S2 (GL) (U.) GE - Gas Envasado """
freq,2503,2380,4571,2503,218,412


In [81]:
# Lista de columnas de interés
columnas_de_interes = ["problema_id", "descripcion", "alias_8_nombre3", "problema_n", "obsitem", "Concaobsitem"]

# Verifica la cantidad de valores nulos por columna
valores_nulos_por_columna = df[columnas_de_interes].isnull().sum()

# Imprime la cantidad de valores nulos por columna
print(valores_nulos_por_columna)

problema_id           0
descripcion          16
alias_8_nombre3      16
problema_n            0
obsitem            2322
Concaobsitem          0
dtype: int64


In [8]:
# Buscar y reemplazar la categoría B1 y B2
categoria_a_buscar = 'B2 - Pérdidas de gas sin piezas dañadas'
categoria_a_reemplazar = 'B1 - Pérdidas de gas con piezas dañadas'

df['problema_n'] = df['problema_n'].replace(categoria_a_buscar, categoria_a_reemplazar)

In [10]:
# Buscar y reemplazar la categoría C2 y C3 por C1 con la descripcion de C2
#se elimina la categoria C2
categoria_a_buscar = 'C2 - Horno Mal funcionamiento'
categoria_a_reemplazar = 'C1 - Horno Mal funcionamiento'

df['problema_n'] = df['problema_n'].replace(categoria_a_buscar, categoria_a_reemplazar)

#se actualiza la categoria C1
categoria_a_buscar = 'C1 - Horno No enciende'
categoria_a_reemplazar = 'C1 - Horno Mal funcionamiento'

df['problema_n'] = df['problema_n'].replace(categoria_a_buscar, categoria_a_reemplazar)

#se elimina la categoria C3
categoria_a_buscar = 'C3 - Horno Se Apaga'
categoria_a_reemplazar = 'C1 - Horno Mal funcionamiento'

#contar = (df['problema_n'] == 'C2 - Horno Mal funcionamiento').sum()
#contar

In [85]:
#Agrupaciones 
df['problema_n'] = df['problema_n'].replace('K - Fallas en encendido electrónico y/o luz de horno', 'K - Fallas en encendido electrónico')
df['problema_n'] = df['problema_n'].replace('B - Pérdidas de gas sin/con piezas quemadas', 'B - Pérdidas de gas – sin/con piezas quemadas')
df['problema_n'] = df['problema_n'].replace('J - Defectos estéticos - Modelos Ac.  inoxidable', 'I - Defectos estéticos')
df['problema_n'] = df['problema_n'].replace('C1 - Horno No enciende', 'C2 - Horno Mal funcionamiento')
df['problema_n'] = df['problema_n'].replace('C3 - Horno Se Apaga', 'C2 - Horno Mal funcionamiento')
df['problema_n'] = df['problema_n'].replace('E - El agua sale con temperatura baja, alta', 'E - El agua sale con temperatura baja / alta')

##Hornallas
df['problema_n'] = df['problema_n'].replace('D4 - Otra Hornalla no enciende', 'D2 - Hornalla Mal funcionamiento')
df['problema_n'] = df['problema_n'].replace('D6 - Otra Hornalla Se Apaga', 'D2 - Hornalla Mal funcionamiento')
df['problema_n'] = df['problema_n'].replace('D3 - Hornalla Se Apaga', 'D2 - Hornalla Mal funcionamiento')
df['problema_n'] = df['problema_n'].replace('D1 - Hornalla No enciende', 'D2 - Hornalla Mal funcionamiento')
df['problema_n'] = df['problema_n'].replace('D5 - Otra Hornalla Mal funcionamiento', 'D2 - Hornalla Mal funcionamiento')

#Elimino los que "no sirven"
# Crear una condición para filtrar los registros que NO contienen los valores especificados
condicion = ~df['problema_n'].isin(['K - Otro problema no mencionado en opciones anteriores', 'J - Otras piezas en mal estado','E - No se apaga el calefón al cerrar la circulación de agua','L - Falta/Falla accesorios Kit MULTIGAS','Q - Problema no mencionado en opciones anteriores','P - Otras piezas en mal estado','H - Piezas se caen, mal fijadas o mal posicionadas','N - Manija rota','H - Accesorios faltantes','I - Otros defectos','G - Piezas faltantes / mal fijadas','F - No apaga','I - Accesorios cambio de GAS','H - Accesorios faltantes','J - Defectos estéticos','G - Piezas faltantes / mal fijadas','K - Problema no mencionado en opciones anteriores'])
df = df[condicion]


In [86]:
# Cuenta las cantidades de clases
clase_counts = df["problema_n"].value_counts()

# Obtiene la lista de clases únicas
clases_unicas = df["problema_n"].unique()

# Cuenta la cantidad de clases únicas
cantidad_clases = len(clases_unicas)

# Imprime la cantidad de clases únicas
print("Cantidad total de clases únicas:", cantidad_clases)

Cantidad total de clases únicas: 26


In [87]:
# Se elimina las filas vacias de los campos descripcion, alias_8_nombre3  y obsitem
df.dropna(subset=['descripcion', 'alias_8_nombre3','obsitem'], inplace=True)

In [88]:
# Concatenar las columnas "descripcion", "alias_8_nombre3" y "obsitem" en una nueva columna "Concaobsitem"
df['Concaobsitem'] = df['descripcion'] + ' ' + df['alias_8_nombre3']+ ' ' + df['obsitem']

In [89]:
df.shape

(15617, 6)

In [90]:
# Lista de columnas de interés
columnas_de_interes = ["problema_id", "descripcion", "alias_8_nombre3", "problema_n", "obsitem", "Concaobsitem"]

# Verifica la cantidad de valores nulos por columna
valores_nulos_por_columna = df[columnas_de_interes].isnull().sum()

# Imprime la cantidad de valores nulos por columna
print(valores_nulos_por_columna)

problema_id        0
descripcion        0
alias_8_nombre3    0
problema_n         0
obsitem            0
Concaobsitem       0
dtype: int64


In [56]:
# Obtener las columnas de diagnósticos y códigos AIS
descripciones = df['Concaobsitem'].tolist()
etiquetas = df['problema_n'].tolist()

In [57]:
for desc in descripciones:
    if not isinstance(desc, str):
        print(f"Elemento {desc} no es una cadena de texto (str).")

In [58]:
# Codificar las etiquetas como etiquetas numéricas
label_encoder = LabelEncoder()
etiquetas_numericas = label_encoder.fit_transform(etiquetas)

In [59]:
# Preprocesamiento de texto
stemmer = SnowballStemmer('spanish')
stop_words = set(stopwords.words('spanish'))

In [60]:
def preprocesar_texto(texto):
    palabras = nltk.word_tokenize(texto.lower())
    palabras = [stemmer.stem(palabra) for palabra in palabras if palabra.isalpha() and palabra not in stop_words]
    return ' '.join(palabras)

In [61]:
descripciones_preprocesadas = [preprocesar_texto(desc) for desc in descripciones]

In [62]:
# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(descripciones_preprocesadas, etiquetas_numericas, test_size=0.2, random_state=42)

In [63]:
# Creación del pipeline para el clasificador basado en texto
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),  # Experimenta con diferentes valores para max_features
    ('clf', SVC(kernel='linear'))  # Experimenta con diferentes kernels
])

In [64]:
# Entrenamiento del modelo
pipeline.fit(X_train, y_train)
scores = cross_val_score(pipeline, X_train, y_train, cv=5) 

In [65]:
print("Accuracy en validación cruzada:", scores.mean())

Accuracy en validación cruzada: 0.7468176381841768


In [66]:
# Evaluación del modelo en el conjunto de prueba
y_pred = pipeline.predict(X_test)
report = classification_report(y_test, y_pred, labels=np.unique(etiquetas_numericas), target_names=label_encoder.inverse_transform(np.unique(etiquetas_numericas)), zero_division=1)
print(report)

                                                                                                   precision    recall  f1-score   support

                                                                    B - Descargas de electricidad       1.00      0.20      0.33        20
                                               B - La llama no enciende al abrir el grifo de agua       0.69      1.00      0.82        29
                                                    B - Pérdidas de gas – sin/con piezas quemadas       0.67      0.17      0.27        12
                                                          B1 - Pérdidas de gas con piezas dañadas       0.42      0.14      0.21        70
                                                          B2 - Pérdidas de gas sin piezas dañadas       0.69      0.76      0.72       188
                              C - La llama se apaga luego de algunos minutos de estar funcionando       0.75      0.46      0.57        13
                          

In [67]:
# Cargar descripciones de problemas para evaluar
archivo_evaluacion = 'C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/textMining/proyecto/TextMining/TextMining/testesco.txt'
lineas = pd.read_csv(archivo_evaluacion, delimiter='\t', header=None, names=['Descripcion'])
lineas

,Descripcion
0,"TERMO 45 L GN (U.) GAS - Termo a gas "" EL EQUI..."
1,"TERMO ELECTRICO EL-55 ELE - Termo eléctrico "" ..."
2,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
3,"TERMO 80 L GN (U.) GAS - Termo a gas "" NO SE M..."
4,"TERMO ELECTRICO EL-90 ELE - Termo eléctrico "" ..."
5,"COCINA CANDOR S2 (GN) (U.) GN - Gas Natural "" ..."
6,COCINA MASTER S2 BL CLASSIC. (GN.) (U.) GN - G...
7,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
8,COCINA MASTER S2 NEG.CLASSIC (GN) (U.) GN - Ga...
9,COCINA MASTER S2 NEG.CLASSIC (GN) (U.) GN - Ga...


In [68]:
resultados = []

for index, row in lineas.iterrows():
    problema = row['Descripcion'].strip()
    problema_preprocesado = preprocesar_texto(problema)
    predicciones = pipeline.predict([problema_preprocesado])
    clase_predicha = label_encoder.inverse_transform(predicciones)[0]
    resultados.append({
        'Problema': problema,
        'Etiqueta de Reclamo': clase_predicha
    })
    

resultados_df = pd.DataFrame(resultados)

In [69]:
archivo_resultado = 'C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/textMining/proyecto/TextMining/TextMining/testresultadoCV_SVCFinal.txt'
resultados_df.to_csv(archivo_resultado, index=False, sep='\t', encoding='utf-8')

print("Resultados guardados en:", archivo_resultado)

Resultados guardados en: C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/textMining/proyecto/TextMining/TextMining/testresultadoCV_SVCFinal.txt


In [70]:
# Leer el archivo excel
dfPrueba = pd.read_csv(archivo_resultado, encoding='utf-8', delimiter='\t')
dfPrueba

,Problema,Etiqueta de Reclamo
0,"TERMO 45 L GN (U.) GAS - Termo a gas "" EL EQUI...",C - Pérdidas de agua
1,"TERMO ELECTRICO EL-55 ELE - Termo eléctrico "" ...",C - Pérdidas de agua
2,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...,E - Puerta de horno - No cierra / Se cae
3,"TERMO 80 L GN (U.) GAS - Termo a gas "" NO SE M...",E - Piloto se apaga
4,"TERMO ELECTRICO EL-90 ELE - Termo eléctrico "" ...",C - Pérdidas de agua
5,"COCINA CANDOR S2 (GN) (U.) GN - Gas Natural "" ...",E - Puerta de horno - No cierra / Se cae
6,COCINA MASTER S2 BL CLASSIC. (GN.) (U.) GN - G...,D2 - Hornalla Mal funcionamiento
7,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...,D2 - Hornalla Mal funcionamiento
8,COCINA MASTER S2 NEG.CLASSIC (GN) (U.) GN - Ga...,D2 - Hornalla Mal funcionamiento
9,COCINA MASTER S2 NEG.CLASSIC (GN) (U.) GN - Ga...,B2 - Pérdidas de gas sin piezas dañadas
